<a href="https://colab.research.google.com/github/JuanAcevedo08/DeepLearning/blob/main/rese%C3%B1a_peliculas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Instalación e importación de librerías y comprobamos versiones

In [ ]:
%%capture
!pip install torch==2.2.0
!pip install torchtext==0.17.2 torchdata==0.7.1 portalocker>=2.0.0

In [ ]:
%%capture
import torch, torchtext, torchdata #Librerías a utilizar

In [ ]:
#Comprbamos las versiones
print(f'Version torch: {torch.__version__}')
print(f'Version torchtext: {torchtext.__version__}')
print(f'Version torchdata: {torchdata.__version__}')

In [ ]:
from torchtext.datasets import IMDB

# 2 Importamos los datos que necesitamos procesarlos(Tokenizacion y vocabulario)


¿Que es tokenizar y que es un vocabulario?




> Tokenizar: Es el proceso mediante se dividen palabras dentro de una oración. Ejemplo: 'Hi how are you?'  el tokenizador se encarga de agarrar los espacios entre la oración para poder dividirlos y que nos de algo como: 'hi', 'how', 'are', 'you'




> Vocabulario: En este se le otorga un numero a cada token , como vimos anterior un token es una palabra de una oración , el vocabulario le otorga un identificador para poder determinar palabras unicas, Ejemplo: {'hi': 0} identificador 0



In [ ]:
#Visualizacion
train_iter = iter(IMDB(split=('train')))
next(train_iter) #Visualizar un ejemplo

In [ ]:
#Importar librerias para tokenizador y vocabulario
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#Creamos el tokenizador
tokenizador = get_tokenizer('basic_english') #Definimos el idioma de los datos

#Instanciamos los datos de entrenamiento para que realice el vocabulario
train_iter = IMDB(split=('train'))

#Creamos una función para que le aplique el tokenizador a cada uno de los comentarios
def tokenizar_comentarios(data):
  for _, text in data:
    yield tokenizador(text)

#Creamos el vocabulario
vocab = build_vocab_from_iterator(tokenizar_comentarios(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
#Probamos el vocabulario
texto = 'The fate of the human is be safe in this world'

#Texto pasado por el vocab
print(vocab(tokenizador(texto)))

# 2.1 Realizar funciones automatizadoras

In [ ]:
#Funciones lambada

#Convertidora a vocabulario
convertidor_vocab = lambda x: vocab(tokenizador(x))

#Ajustador clase | Este se da , debido a que la función de costo ya que las salidas son [0...nclases] y la funcion de costo compara con clase real [1...n] hay un indice menos entonces hay que bajarle 1
ajuste_numerico = lambda x: int(x) - 1

# 3 Función collate



---


Cuando manejamos grandes volumnes de datos durante muchas veces es necesario tener un data loader que divida nuestros dato enbatches para que pase batches en vez de todos los datos enteros entonces en estos dataloaders cuando pasamos nuestros datos es mejor para el modelo que reciba no solo los datos ya procesados sino ademas que los reciba en un densor de una dimension par que sea mas facil realizar este proceso para lo que vamos a necesitar el tensor con tensores, el tensor con labels reales y un offsets ya que como lo que vamos a hacer es concatenar tensores en uno solo , el offsets indicará donde va inciar la oracion

In [ ]:
#Fijamos dispotivo que van a tener los tensores
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #Es muy importante ya que los tensores que creemos tienen que permanecer en el mismo tipo de dato y dispositivo

#Creación de la función

def collate_(batch):
  """
  Parametro: un lote de todos los datos para ser procesados
  durante este proceso , separa el lote en texto y etiquetas para poder procesar tanto texto como etiqueta agregando en una lista cada etiqueta y texto procesado y se establece que todos esten el mismo tipo de dato
  etiquetas_: es un tensor que contiene otro tensor con las tiquetas en enteros
  indicadores_: contiene un tensor con las sumass de las longiudes de los textos para indicar donde va iniciar cada frase
  textos_: tiene un tensor que contiene una concatenacion de  enteros donde cada entero representa una palabra del texto

  return: 3 tensores en el mismo dispotivio y mismo tipo de tensor
  """
  etiquetas_ = []
  indicadores_ = []
  textos_ = []

  for (etiqueta_ , texto_) in batch:
    etiquetas_.append(ajuste_numerico(etiqueta_))
    texto_torch = torch.tensor(convertidor_vocab(texto_), dtype=torch.int64)
    textos_.append(texto_torch)
    indicadores_.append(texto_torch.size(0))

  etiquetas_ = torch.tensor(etiquetas_, dtype=torch.int64)
  indicadores_ = torch.tensor([0] + torch.tensor(indicadores_[:-1]).cumsum(dim=0).tolist(), dtype=torch.int64)

  textos_ = torch.cat(textos_)

  return etiquetas_.to(device), indicadores_.to(device) , textos_.to(device)

# 4 Creación de modelo clasificador de texto

En este modelo tenemos 3 diferentes entradas donde dos de ellas nos van a servir para el embedding , el embedding se encarga de utilizar el vocabulario para crear vectores de densidad que nos puedan representar similitudes entre palabras para esto indico la cantidad de palabras en vocabulario y el tamaño de los vectores a realizar luego de eso tenemos un atributo de normalizacion que se va encargar de estandarizar los valroes a las mismas escalas usando la formula conocida x - u / o , de esa manera evita que el modelo se confunda con valores muy altos y finalmente una funcion de decisión que se encarga de prpoyectar cada una de las entradas a la cantidad de clase que tenemos para definir sus valores y tomar la decisión

***Forward:***

En este empezamos con el embedgin este es un bag que se va encargar de ademas de sus atributos fijos tiene atributos dinamicos , ya que estos son para cuando tengo este tipo de tensores concatenados entonces me toca indicar de donde a donde va realizar los embeding , luefo de realizar los embeds por cada oracion dentro del tensor se encarga de promediar , sumar o agarrar el valor maximo de los embds para definir un solo embd por oracion y finalmente normalizarlo, luego de eso lo pasa a una fucnion de activacion que va determinar su valor para pasarlo a conclusion de que clase es

In [ ]:
class ModelTextClassifier(torch.nn.Module):
  def __init__(self, vocab_size, embed_dim, num_clases):
    super(ModelTextClassifier, self).__init__()

    self.embd = torch.nn.EmbeddingBag(vocab_size, embed_dim)
    self.norm = torch.nn.BatchNorm1d(embed_dim)
    self.fc = torch.nn.Linear(embed_dim, num_clases)

  def forward(self, text, offsets):
    embed = self.embd(text, offsets)
    norm = self.norm(embed)
    activacion = torch.nn.functional.relu(norm)
    return self.fc(activacion)

In [ ]:
#Revisar estructura, parametros y clases del modelo
temp_iter = IMDB(split='test')
n_clases = len(set([label for (label, text) in temp_iter]))
modelo_clf = ModelTextClassifier(len(vocab), 200, n_clases).to(device) #Poner el modelo en el mismo dispotivo de los tensores

#Función para revisar catnidad de parametros entrenables
def parametros_entrenables(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Cantidad de clases {n_clases} ')
print(f'Estructura del modelo:\n {modelo_clf}')
print(f'Cantidad de parametros a entrenar {parametros_entrenables(modelo_clf):,}')

#5 Función entrenamiento  

In [ ]:
#Crear una funcion de entrenamiento para el modelo

def funcion_entrenamiento(modelo, dataloader):
  #Activar modeo de entrenamiento
  modelo.train()
  #Creación de variables
  precision_ = 0
  perdida_ = 0
  etiquetas_ = 0

  for (etiqueta , indicador , texto ) in dataloader:
    #Limpiar gradientes
    optimizador.zero_grad()
    #Realizar una preddición
    prediccion = modelo(texto, indicador)
    #Calcular perdida
    perdida = criterio(prediccion, etiqueta)
    #Calcular aciertos
    aciertos = (prediccion.argmax(1) == etiqueta).sum()
    #Realizar backward (calculo de gradientes)
    perdida.backward()
    #Cortar los parametros a 0.1 para que no sea un gradiente muy grande
    torch.nn.utils.clip_grad_norm_(modelo.parameters(), 0.1)
    #Ajustar los pesos
    optimizador.step()
    #Ajustar las variables de información
    precision_ += aciertos.item()
    perdida_ += perdida.item()
    etiquetas_ += etiqueta.size(0)

  precision_promedio = precision_ / etiquetas_
  perdida_promedio = perdida_ / etiquetas_

  return precision_promedio, perdida_promedio

# 6 Funcion de evaluacion

In [ ]:
def funcion_evaluacion(modelo, dataloader):
  #Ajusta en modo evaluacion
  modelo.eval()
  #Crear variables para metricas
  precision_ = 0
  perdidas_ = 0
  etiquetas_ = 0

  #Retirar el gradiente para realizar evaluaciones sin gastar memoria
  with torch.no_grad():
    for i, (etiqueta_ , indicadores_, textos_) in enumerate(dataloader):
      #Realizar preddicion con los datos de validacion
      prediccion = modelo(textos_, indicadores_)
      #Consultar perdida
      perdida = criterio(prediccion, etiqueta_)
      #Crear precision
      precision = (prediccion.argmax(1) == etiqueta_).sum()

      #Ajustar variaables - metricas
      precision_ += precision.item()
      perdidas_ += perdida.item()
      etiquetas_ += etiqueta_.size(0)

  #Calcular los promedios
  precision_promedio = precision_ / etiquetas_
  perdida_promedio = perdidas_ / etiquetas_


  return precision_promedio, perdida_promedio

# 7 Creacion de funcion de perdida, optimzador, separacion de datos y dataloaders

In [ ]:
#Hyper Parametros
epocas = 10
lotes = 64
lr = 5e-3
momentum = 0.9
#Funcion de perdida
criterio = torch.nn.CrossEntropyLoss()
#Optimzador
optimizador = torch.optim.SGD(modelo_clf.parameters(), lr=lr, momentum=momentum)

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

# Cargar solo el split de test que tiene ambas clases , ya que train no tiene
datos_completos = iter(IMDB(split='test'))
datos_completos = to_map_style_dataset(datos_completos)

# Dividir en: 76% entrenamiento, 19% validación, 5% prueba
total = len(datos_completos)
train_size = int(0.76 * total)
val_size = int(0.19 * total)
test_size = total - train_size - val_size

separacion_entrenamiento, separacion_validacion, separacion_prueba = random_split(
    datos_completos,
    lengths=[train_size, val_size, test_size]
)

# Crear dataloaders
dataloader_entrenamiento = DataLoader(separacion_entrenamiento, lotes, shuffle=True, collate_fn=collate_)
dataloader_valdiacion = DataLoader(separacion_validacion, lotes, shuffle=True, collate_fn=collate_)
dataloader_prueba = DataLoader(separacion_prueba, lotes, shuffle=True, collate_fn=collate_)

# 8 Realizar entrenamiento y prueba


In [ ]:
#Crear una variable que tome el mejor promeedio de validaicon de perdida
mejor_perdida_validacion = float('inf')

#Realizar entrenamiento
for epoca in range(1, epocas + 1 ):
  #Empezar a entrenar y recibir informacion
  precision_promedio_et, perdida_promedio_et = funcion_entrenamiento(modelo_clf, dataloader_entrenamiento)
  #Empear a validar y recibir informacion
  precision_promedio_ev, perdida_promedio_ev = funcion_evaluacion(modelo_clf, dataloader_valdiacion)

  print(f'Época {epoca}/{epocas}:')
  print(f'  Train -> Pérdida: {perdida_promedio_et:.4f}, Precisión: {precision_promedio_et:.4f}')
  print(f'  Val   -> Pérdida: {perdida_promedio_ev:.4f}, Precisión: {precision_promedio_ev:.4f}')

  if perdida_promedio_ev < mejor_perdida_validacion:
    mejor_perdida_validacion = perdida_promedio_ev
    torch.save(modelo_clf.state_dict(), 'mejor_modelo_eval.pt')
    print(f'MOdelo guardado')

In [ ]:
precision_promedio_pb, perdida_promedio_pb = funcion_evaluacion(modelo_clf, dataloader_prueba)

print(f'Precision_prueba: {precision_promedio_pb}')
print(f'Perdida_prueba: {perdida_promedio_pb}')

# 9 Realizar inferencia

In [ ]:
#Mapeo de las clases a clasificar
clases = ['neg', 'pos']
#Realizar función de predicción con datos reales
def prediccion(texto, modelo):
  #Ajustar modo inferencia
  with torch.inference_mode():
    #Transformacion del texto
    entrada = torch.tensor(convertidor_vocab(texto), device=device)
    #Realiza prediccion
    salida = modelo(entrada, torch.tensor([0] , device=device, dtype=torch.int64))

    return salida.argmax(1).item()

# 10 Crear funcion para probar ejemplos


In [ ]:
#Simular una base de datos falsa
db_falsa = [[
     # NEGATIVAS (0)
    'The movie was boring and too long, I almost fell asleep.',
    'Terrible customer service, I will never return.',
    'The product broke after one use, very disappointing.',
    'The food was cold and tasteless, not worth the money.',
    'Worst experience ever, I regret buying this.'],

    [# POSITIVAS (1)
    'This movie was amazing, I loved every second of it!',
    'The product exceeded my expectations, highly recommended.',
    'Great service and friendly staff, will come back again.',
    'Fantastic quality for the price, I am very satisfied.',
    'An excellent experience, everything was perfect.']
]

#Realizar la función para probar ejemplos
def pruebas(db, modelo, pred_fn):
  aciertos = 0
  espacio_muestral = 0
  for i, v in enumerate(db):
    print('*' * 200)
    print(f'Entrando en sección reseñas {clases[i]}')
    for texto in v:
      valor = pred_fn(texto, modelo)
      print(f'Texto: \n {texto}')
      print(f'Predicción {clases[valor]} | Clase a la que pertenece {clases[i]}')
      if clases[valor] == clases[i]:
        aciertos += 1
      espacio_muestral += 1

In [ ]:
#Probar los ejemplos

pruebas(db_falsa, modelo_clf, prediccion)

# 11 cargar el modelo con mejor precision para val

In [ ]:
checkpoint = torch.load('/content/mejor_modelo_eval.pt') #Instanciamos un checkpoint

In [ ]:
temp_iter = IMDB(split='test') #Crear el iterador
#Variables necesarias para el modelo
n_clases = len(set([label for (label, text) in temp_iter]))
vocab_size = len(vocab)
embd_dimn = 200
#Crear el nuevo modelo y poner las clases
# Versión simplificada para solo inferencia
modelo_clfs_2 = ModelTextClassifier(len(vocab), 200, 2).to(device)
modelo_clfs_2.load_state_dict(torch.load('/content/mejor_modelo_eval.pt'))
modelo_clfs_2.eval()

# Probar
text = "This film was a complete waste of time. The acting was terrible and the plot made no sense."
value = prediccion(text, modelo_clfs_2)
print(f'Clase real: neg | Clase predicha: {clases[value]}')

# 12 Cargar el modelo para subir a huggin face

In [ ]:
#Cargar los parametros del modelo
modelo_pesos = modelo_clfs_2.state_dict()
optimizador_dict = optimizador.state_dict()
checkpoint_ = {
    'Modelo': modelo_pesos,
    'optimizador': optimizador_dict,
    'epocas': epocas,
    'learning_rate': lr,
    'embd_size': embd_dimn,
    'vocab_size': vocab_size,
    'batch_size': lotes
}

torch.save(checkpoint_, "./model_paraemetrs.pth")

In [ ]:
%%capture
!pip install hugginface_hub

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.create_repo(repo_id="platzi/reviews_imbd-juan-acevedo")

In [ ]:
#Crear repo para mi
api.create_repo(repo_id="reviews_imbd")

In [ ]:
#Subir el contenido al repo
api.upload_file(path_or_fileobj='./model_paraemetrs.pth', path_in_repo='model_paraemetrs.pth', repo_id='platzi/reviews_imbd-juan-acevedo')
api.upload_file(path_or_fileobj='./model_paraemetrs.pth', path_in_repo='model_paraemetrs.pth', repo_id='JuanAcevedo/reviews_imbd')